In [1]:
from adaptive_time.samplers import AdaptiveQuadratureSampler

import numpy as np

In [2]:
seed = 42
np.random.seed(seed)

In [3]:
horizon = 20000
dt = 1
num_steps = horizon
tolerance_init = 0.01

sampler = AdaptiveQuadratureSampler(
    dt=dt,
    num_steps=num_steps,
    tolerance_init=tolerance_init,
    update_when_best=False,
)

In [4]:
def foo(xs,tol,level):
    c=int(np.floor(len(xs)/2))
    #print(xs, c)
    f = lambda xs: len(xs)*(xs[0]+xs[-1])/2 if len(xs) else 0
    if abs(f(xs) - (r:=f(xs[:c]) + f(xs[c:]))) < tol: return 1, r, [level+c]
    else: 
        x, a, cs_a = foo(xs[:c],tol/2, level)
        y, b, cs_b = foo(xs[c:],tol/2, level+c)
        return x+y+1, a+b, cs_a + cs_b + [level+c]

In [5]:
for _ in range(1000):
    traj = np.zeros(horizon)
    change_at = np.random.randint(horizon)
    traj[0:change_at] = -1
    trajs = [{"rews": traj}]
    sample_times, total_seg, num_calls = sampler.adapt(trajs)
    calls, approx_sum, ts = foo(traj, tolerance_init, 0)

    print("====")
    print("total_rews: {}, reward = 0 at: {}".format(sum(traj), change_at))
    print("mine: num_calls {} sum {} idxes {} len {}".format(num_calls, total_seg, sample_times, len(sample_times)))
    print("foo: num_calls {} sum {} idxes {} len {}".format(calls, approx_sum, np.unique(ts), len(np.unique(ts))))

====
total_rews: -15795.0, reward = 0 at: 15795
mine: num_calls 23 sum -15795.0 idxes [ 5000 10000 12500 15000 15312 15625 15703 15781 15785 15790 15795 15800
 15810 15820 15839 15859 15898 15937 16093 16250 16875 17500 18750] len 23
foo: num_calls 23 sum -15795.0 idxes [ 5000 10000 12500 15000 15312 15625 15703 15781 15785 15790 15795 15800
 15810 15820 15839 15859 15898 15937 16093 16250 16875 17500 18750] len 23
====
total_rews: -860.0, reward = 0 at: 860
mine: num_calls 27 sum -860.0 idxes [  312   625   703   781   820   859   860   861   862   863   865   868
   873   878   888   898   917   937  1093  1250  1875  2500  3750  5000
  7500 10000 15000] len 27
foo: num_calls 27 sum -860.0 idxes [  312   625   703   781   820   859   860   861   862   863   865   868
   873   878   888   898   917   937  1093  1250  1875  2500  3750  5000
  7500 10000 15000] len 27
====
total_rews: -5390.0, reward = 0 at: 5390
mine: num_calls 15 sum -5390.0 idxes [ 2500  5000  5156  5312  5390  5468 

In [10]:

traj = np.linspace(-1,1,20000)
traj = traj ** 2
trajs = [{"rews": traj}]
sample_times, total_seg, num_calls = sampler.adapt(trajs)
calls, approx_sum, ts = foo(traj, tolerance_init, 0)

print("====")
print("total_rews: {}, reward = 0 at: {}".format(sum(traj), change_at))
print("mine: num_calls {} sum {} idxes {} len {}".format(num_calls, total_seg, sample_times, len(sample_times)))
print("foo: num_calls {} sum {} idxes {} len {}".format(calls, approx_sum, np.unique(ts), len(np.unique(ts))))

====
total_rews: 6667.333366668338, reward = 0 at: 19891
mine: num_calls 2047 sum 6667.33565169683 idxes [    9    19    29 ... 19971 19981 19991] len 2047
foo: num_calls 2047 sum 6667.33565169683 idxes [    9    19    29 ... 19970 19980 19990] len 2047
